In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import pickle
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

from torchvision.models import resnet101,mobilenet_v2
import sys
import timm
sys.path.insert(0, 'timm==0.4.5')
#import timm #gets version 2
print(f"Timm: {timm.__version__}")
print(f"Torch: {torch.__version__}")

Timm: 0.4.5
Torch: 1.9.0+cu102


In [2]:
IMG_SIZE = 224
use_cuda = torch.cuda.is_available()
batch_size=2

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

test_dir_1 = r'C:\Users\HP\SLR\crossVal\Data\fold_1_c\test_files'
test_dir_2 = r'C:\Users\HP\SLR\crossVal\Data\fold_2_c\test_files'
test_dir_3 = r'C:\Users\HP\SLR\crossVal\Data\fold_3_c\test_files'
test_dir_4 = r'C:\Users\HP\SLR\crossVal\Data\fold_4_c\test_files'
test_dir_5 = r'C:\Users\HP\SLR\crossVal\Data\fold_5_c\test_files'

kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}

test_dataset_1 = datasets.ImageFolder(root=test_dir_1, transform=test_transforms)
test_loader_1  = torch.utils.data.DataLoader(test_dataset_1, batch_size=batch_size, shuffle=False, **kwargs) 

test_dataset_2 = datasets.ImageFolder(root=test_dir_2, transform=test_transforms)
test_loader_2  = torch.utils.data.DataLoader(test_dataset_2, batch_size=batch_size, shuffle=False, **kwargs) 

test_dataset_3 = datasets.ImageFolder(root=test_dir_3, transform=test_transforms)
test_loader_3  = torch.utils.data.DataLoader(test_dataset_3, batch_size=batch_size, shuffle=False, **kwargs) 

test_dataset_4 = datasets.ImageFolder(root=test_dir_4, transform=test_transforms)
test_loader_4  = torch.utils.data.DataLoader(test_dataset_4, batch_size=batch_size, shuffle=False, **kwargs) 

test_dataset_5 = datasets.ImageFolder(root=test_dir_5, transform=test_transforms)
test_loader_5  = torch.utils.data.DataLoader(test_dataset_5, batch_size=batch_size, shuffle=False, **kwargs) 

print(len(test_dataset_1))
print(len(test_dataset_2))
print(len(test_dataset_3))
print(len(test_dataset_4))
print(len(test_dataset_5))

507
507
507
507
507


In [3]:
testing_set = [test_dataset_1, test_dataset_2, test_dataset_3, test_dataset_4, test_dataset_5]
test_dirs = [test_dir_1, test_dir_2, test_dir_3, test_dir_4, test_dir_5]

In [4]:
new_model = torch.load(r'C:\Users\HP\SLR\models\affectnet_emotions\enet_b0_7.pt')
new_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0

In [5]:
class_to_idx = {'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
idx_to_class = {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

In [6]:
from sklearn.metrics import classification_report

targets = ['anger','disgust','fear','happy','neutral','sad','surprise']
k = 1

for test_dir in test_dirs:
    
    print('Fold ' + str(k))

    device = 'cuda'
    y_val,y_scores_val=[],[]
    #new_model.eval()
    for class_name in tqdm(os.listdir(test_dir)):
        if class_name in class_to_idx:
            class_dir=os.path.join(test_dir,class_name)
            y=class_to_idx[class_name]
            for img_name in os.listdir(class_dir):
                filepath=os.path.join(class_dir,img_name)
                img = Image.open(filepath)
                img_tensor = test_transforms(img)
                img_tensor.unsqueeze_(0)
                scores = new_model(img_tensor.to(device))
                scores=scores[0].data.cpu().numpy()
                #print(scores.shape)
                y_scores_val.append(scores)
                y_val.append(y)

    y_scores_val=np.array(y_scores_val)
    y_val=np.array(y_val)
    #print(y_scores_val.shape,y_val.shape)
    
    # Accuracy & Sensitivity per class
    y_pred=np.argmax(y_scores_val,axis=1)
    acc=100.0*(y_val==y_pred).sum()/len(y_val)
    print(acc)

    for i in range(y_scores_val.shape[1]):
        _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
        print('%s acc: %f' %(idx_to_class[i],100*_val_acc))
        
    
    #Classification Report 
    target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
    print(classification_report(y_val, y_pred, target_names=target_names))
    
    k = k + 1

Fold 1


  0%|          | 0/7 [00:00<?, ?it/s]

47.14003944773175
anger acc: 33.333333
disgust acc: 54.545455
fear acc: 50.000000
happy acc: 37.500000
neutral acc: 80.645161
sad acc: 11.764706
surprise acc: 63.513514
              precision    recall  f1-score   support

       Anger       0.81      0.33      0.47       144
     Disgust       0.30      0.55      0.38        55
        Fear       0.22      0.50      0.31        54
       Happy       0.43      0.38      0.40        24
     Neutral       0.46      0.81      0.59        31
     Sadness       0.40      0.12      0.18        51
    Surprise       0.70      0.64      0.67       148

    accuracy                           0.47       507
   macro avg       0.47      0.47      0.43       507
weighted avg       0.58      0.47      0.48       507

Fold 2


  0%|          | 0/7 [00:00<?, ?it/s]

49.30966469428008
anger acc: 47.826087
disgust acc: 50.000000
fear acc: 10.204082
happy acc: 56.818182
neutral acc: 44.444444
sad acc: 22.535211
surprise acc: 74.509804
              precision    recall  f1-score   support

       Anger       0.83      0.48      0.61        92
     Disgust       0.23      0.50      0.31        44
        Fear       0.08      0.10      0.09        49
       Happy       0.69      0.57      0.62        44
     Neutral       0.44      0.44      0.44        54
     Sadness       0.62      0.23      0.33        71
    Surprise       0.63      0.75      0.68       153

    accuracy                           0.49       507
   macro avg       0.50      0.44      0.44       507
weighted avg       0.56      0.49      0.50       507

Fold 3


...

43.58974358974359
anger acc: 45.555556
disgust acc: 58.333333
fear acc: 31.632653
happy acc: 68.421053
neutral acc: 54.166667
sad acc: 37.974684
surprise acc: 44.295302
              precision    recall  f1-score   support

       Anger       0.63      0.46      0.53        90
     Disgust       0.13      0.58      0.21        24
        Fear       0.28      0.32      0.30        98
       Happy       0.72      0.68      0.70        19
     Neutral       0.48      0.54      0.51        48
     Sadness       0.79      0.38      0.51        79
    Surprise       0.59      0.44      0.51       149

    accuracy                           0.44       507
   macro avg       0.52      0.49      0.47       507
weighted avg       0.54      0.44      0.46       507

Fold 4


  0%|          | 0/7 [00:00<?, ?it/s]

49.90138067061144
anger acc: 50.000000
disgust acc: 55.172414
fear acc: 33.802817
happy acc: 60.416667
neutral acc: 36.111111
sad acc: 15.000000
surprise acc: 65.641026
              precision    recall  f1-score   support

       Anger       0.79      0.50      0.61        68
     Disgust       0.17      0.55      0.26        29
        Fear       0.32      0.34      0.33        71
       Happy       0.56      0.60      0.58        48
     Neutral       0.26      0.36      0.30        36
     Sadness       0.60      0.15      0.24        60
    Surprise       0.72      0.66      0.69       195

    accuracy                           0.50       507
   macro avg       0.49      0.45      0.43       507
weighted avg       0.58      0.50      0.51       507

Fold 5


  0%|          | 0/7 [00:00<?, ?it/s]

38.658777120315584
anger acc: 34.782609
disgust acc: 60.000000
fear acc: 25.000000
happy acc: 43.181818
neutral acc: 57.692308
sad acc: 10.344828
surprise acc: 50.609756
              precision    recall  f1-score   support

       Anger       0.58      0.35      0.43       115
     Disgust       0.16      0.60      0.25        35
        Fear       0.12      0.25      0.16        36
       Happy       0.83      0.43      0.57        44
     Neutral       0.24      0.58      0.34        26
     Sadness       0.56      0.10      0.17        87
    Surprise       0.66      0.51      0.57       164

    accuracy                           0.39       507
   macro avg       0.45      0.40      0.36       507
weighted avg       0.54      0.39      0.41       507



# Accuracy

In [7]:
from sklearn.metrics import accuracy_score

acc=100.0*(y_val==y_pred).sum()/len(y_val)
print('Accuracy:',acc)

Accuracy: 38.658777120315584


In [8]:
# Weighted accuracy

from sklearn.metrics import f1_score
w_acc = f1_score(y_val, y_pred, average='weighted')
print(w_acc)

0.40894046614331425


In [9]:
from sklearn.metrics import classification_report
clas_report = classification_report(y_val, y_pred, labels=None, target_names=None, sample_weight=None)
print(clas_report)

              precision    recall  f1-score   support

           0       0.58      0.35      0.43       115
           1       0.16      0.60      0.25        35
           2       0.12      0.25      0.16        36
           3       0.83      0.43      0.57        44
           4       0.24      0.58      0.34        26
           5       0.56      0.10      0.17        87
           6       0.66      0.51      0.57       164

    accuracy                           0.39       507
   macro avg       0.45      0.40      0.36       507
weighted avg       0.54      0.39      0.41       507



# Confusion Matrix

In [10]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_val, y_pred, normalize='true')
print(idx_to_class.values())
print(cm)

dict_values(['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'])
[[0.34782609 0.33913043 0.07826087 0.         0.11304348 0.03478261
  0.08695652]
 [0.         0.6        0.22857143 0.11428571 0.         0.02857143
  0.02857143]
 [0.02777778 0.41666667 0.25       0.         0.02777778 0.02777778
  0.25      ]
 [0.02272727 0.22727273 0.         0.43181818 0.09090909 0.
  0.22727273]
 [0.07692308 0.03846154 0.07692308 0.         0.57692308 0.
  0.23076923]
 [0.11494253 0.43678161 0.08045977 0.         0.18390805 0.10344828
  0.08045977]
 [0.09146341 0.05487805 0.26219512 0.         0.07926829 0.00609756
  0.50609756]]


# Sensitivity

In [21]:
# Sensitivity per class

from sklearn.metrics import recall_score
sensitivity_per_class = recall_score(y_val,y_pred,average=None)
print(sensitivity_per_class)

[0.49635036 0.67741935 0.36842105 0.47368421 0.47619048 0.18644068
 0.73053892]


In [25]:
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )


Sensitivity :  0.6238532110091743
